In [102]:
import pandas as pd
import numpy as np
import random

In [103]:
def read_data(file):
    data = pd.read_excel(file, header=None)
    return data

In [104]:
data = read_data("JSSP-problems/basic-problem.xlsx")
data

,0,1,2
0,3,2,2
1,2,4,1
2,0,4,3


# CHROMOSOMA

Representat com una llista de tuples, on gada gen es una tupla. La primera posició de la tupla indica el job i la segona posició indica la màquina. 

In [105]:
params = {"population_size": 10, 
          "mutation_rate": 0.1, 
          "crossover_rate": 0.8, 
          "max_generations": 100}

In [106]:
def generate_inital_population(data, population_size):
    jobs = data.shape[0]
    machines = data.shape[1]
    population = []
    for _ in range(population_size):
        chromosome = []
        for job in range(jobs):
            for machine in range(machines):
                chromosome.append((job, machine))
        random.shuffle(chromosome)
        population.append(chromosome)
    return population


In [107]:
def fitness(chromosome, data):
    pass

In [108]:
def selection(population, fitness_values, tournament_size):
    pass

In [109]:
def crossover(parent1, parent2):
    pass

In [110]:
def mutation(chromosome, mutation_rate):
    pass

In [113]:
def gen_algorithm(data, params):
    population_size = params["population_size"]
    mutation_rate = params["mutation_rate"]
    crossover_rate = params["crossover_rate"]
    max_generations = params["max_generations"]

    population = generate_inital_population(data, population_size)
    return population

In [114]:
gen_algorithm(data, params)

[[(2, 0), (2, 1), (1, 2), (0, 2), (0, 0), (2, 2), (1, 1), (1, 0), (0, 1)],
 [(1, 1), (2, 0), (0, 0), (1, 0), (0, 1), (2, 1), (2, 2), (1, 2), (0, 2)],
 [(1, 1), (1, 0), (1, 2), (2, 2), (2, 1), (0, 1), (2, 0), (0, 0), (0, 2)],
 [(2, 1), (0, 2), (2, 2), (1, 2), (0, 0), (2, 0), (0, 1), (1, 0), (1, 1)],
 [(1, 2), (0, 0), (0, 2), (2, 2), (2, 0), (1, 1), (1, 0), (2, 1), (0, 1)],
 [(2, 2), (1, 1), (0, 1), (1, 0), (0, 2), (2, 1), (0, 0), (1, 2), (2, 0)],
 [(2, 0), (1, 0), (1, 1), (2, 2), (1, 2), (0, 2), (0, 0), (0, 1), (2, 1)],
 [(1, 1), (0, 1), (0, 0), (1, 0), (2, 2), (2, 1), (1, 2), (0, 2), (2, 0)],
 [(0, 1), (2, 1), (1, 0), (0, 2), (2, 0), (2, 2), (0, 0), (1, 1), (1, 2)],
 [(2, 1), (0, 2), (2, 0), (1, 2), (0, 0), (2, 2), (1, 1), (0, 1), (1, 0)]]